In [1]:
from tableaudocumentapi import Datasource
import numpy as np
import pandas as pd
import csv, os , datetime ,pprint
from tableaudocumentapi import Workbook
from tableaudocumentapi import *

### List of actions: 
    1. Work out a workbook list 
    2. Functionalise the workbook to datasources functions 
    3. Functionalise the datasources to DF 
    3. Collapse dfs into single df 

In [2]:
os.chdir('document-api-python-master\\samples\\replicate-workbook\\')
print(os.listdir())

['.ipynb_checkpoints', 'databases.csv', 'replicate_workbook.py', 'sample-superstore.twb', 'World Happiness.twb']


In [3]:
[print(n) for n in os.listdir() if n.endswith(('.twb'))]

sample-superstore.twb
World Happiness.twb


[None, None]

In [4]:
todo = [n for n  in os.listdir() if n.endswith(('.twb'))]
print(type(todo))
print(todo)

<class 'list'>
['sample-superstore.twb', 'World Happiness.twb']


In [5]:
def datasources2df(wbf,rds):
    ds1 = pd.DataFrame([f.__dict__.values() for f in rds.fields.values()], columns = [f.__dict__.keys() for f in rds.fields.values()][1])
    ds2 = pd.DataFrame([f for f in rds.fields.keys()],columns = ['name'])
    ds = ds2.join(ds1)
    #Clean Column Names:
    ds.columns = [col.replace('_','') for col in ds.columns]
    #Clean Variable Names: 
    ds['name'] = ds['name'].str.replace('[','').str.replace(']','')
    #Import workbook information:
    ds['datasoure_caption'] = str(rds.caption)
    ds['datasoure_connections'] = str(rds.connections)
    ds['datasoure_name'] = str(rds.name)
    ds['datasoure_version'] = str(rds.version)
    ds['from_wb'] = str(Workbook(wbf).filename)
    return ds

In [6]:
list4dfs = {}
for n, twb in enumerate(todo): 
    for ds in Workbook(twb).datasources:
        print ("Name : " ,Workbook(twb).filename)
        print (ds, " : " ,ds.name , '\n',ds.caption)
        print ([f.__dict__.keys() for f in ds.fields.values()][1])
        print ("ID : " ,n+1)
        list4dfs[n+1] = datasources2df(twb, ds)
        print (list4dfs[n+1].shape)
#ds

Name :  sample-superstore.twb
<tableaudocumentapi.datasource.Datasource object at 0x00000175ACB76CC0>  :  sqlserver.187oiqw1ls5tey1df9op71phda18 
 Orders (sample_superstore)
dict_keys(['_id', '_caption', '_datatype', '_role', '_type', '_alias', '_calculation', '_description', '_aggregation', '_worksheets'])
ID :  1
(25, 16)
Name :  World Happiness.twb
<tableaudocumentapi.datasource.Datasource object at 0x00000175ACC38CF8>  :  federated.0ne8o6b1yp56zg1cblpoy03mpgbl 
 WorldHappniessCountries2015to2017
dict_keys(['_id', '_caption', '_datatype', '_role', '_type', '_alias', '_calculation', '_description', '_aggregation', '_worksheets'])
ID :  2
(40, 16)


In [7]:
totdf = pd.concat(list4dfs.values(), ignore_index=True)
print(totdf.shape)
pprint.pprint(totdf['datasoure_connections'].sample(5))

(65, 16)
41    ['<Connection server='' dbname='None' @ 0x175a...
0     ['<Connection server='superstore.sample.db' db...
27    ['<Connection server='' dbname='None' @ 0x175a...
43    ['<Connection server='' dbname='None' @ 0x175a...
25    ['<Connection server='' dbname='None' @ 0x175a...
Name: datasoure_connections, dtype: object


In [8]:
print(totdf['datasoure_connections'][3:45:6])

3     ['<Connection server='superstore.sample.db' db...
9     ['<Connection server='superstore.sample.db' db...
15    ['<Connection server='superstore.sample.db' db...
21    ['<Connection server='superstore.sample.db' db...
27    ['<Connection server='' dbname='None' @ 0x175a...
33    ['<Connection server='' dbname='None' @ 0x175a...
39    ['<Connection server='' dbname='None' @ 0x175a...
Name: datasoure_connections, dtype: object


In [9]:
print(list4dfs.keys())
type(list4dfs)

dict_keys([1, 2])


dict

In [28]:
print(totdf.info())
print(totdf.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 16 columns):
name                     65 non-null object
id                       65 non-null object
caption                  5 non-null object
datatype                 65 non-null object
role                     20 non-null object
type                     20 non-null object
alias                    27 non-null object
calculation              5 non-null object
description              0 non-null object
aggregation              43 non-null object
worksheets               65 non-null object
datasoure_caption        65 non-null object
datasoure_connections    65 non-null object
datasoure_name           65 non-null object
datasoure_version        65 non-null object
from_wb                  65 non-null object
dtypes: object(16)
memory usage: 8.2+ KB
None
Index(['name', 'id', 'caption', 'datatype', 'role', 'type', 'alias',
       'calculation', 'description', 'aggregation', 'worksheets',
       'datasou

In [31]:
outdf = totdf.worksheets.apply(list).apply(pd.Series) \
    .merge(totdf, left_index = True, right_index = True) \
    .drop(["worksheets"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'caption', 'datatype', 'role', 'type', 'alias', 'calculation', 
                     'description', 'aggregation', 'datasoure_caption', 'datasoure_connections', 'datasoure_name',
                       'datasoure_version', 'from_wb'], value_name = "worksheet")
print(outdf.info())
print(outdf.columns)
pprint.pprint(outdf.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 17 columns):
name                     130 non-null object
id                       130 non-null object
caption                  10 non-null object
datatype                 130 non-null object
role                     40 non-null object
type                     40 non-null object
alias                    54 non-null object
calculation              10 non-null object
description              0 non-null object
aggregation              86 non-null object
datasoure_caption        130 non-null object
datasoure_connections    130 non-null object
datasoure_name           130 non-null object
datasoure_version        130 non-null object
from_wb                  130 non-null object
variable                 130 non-null object
worksheet                21 non-null object
dtypes: object(17)
memory usage: 17.3+ KB
None
Index(['name', 'id', 'caption', 'datatype', 'role', 'type', 'alias',
       'calculation', '